### Energieversorgung Vertiefung Hausübung 1: Zuverlässigkeitskennzahlen

In [27]:
import pandas as pd
from tabulate import tabulate
import numpy as np

In [28]:
# Eingabe Matrikelnummer
#MatrNr = 12012289
MatrNr = 52102155
# MatrNr = 12143819
# MatrNr = 51911209
# MatrNr = 12012289

# Berechnung IPS
def calculation_ips(Matrikelnummer):
    q = sum(int(digit) for digit in str(Matrikelnummer))
    return ((q * 5) % 40) * 10**-4

IPS = calculation_ips(MatrNr)
print(IPS)

# Allgemeine Funktionen
def process_values(val):
    return (0, 1) if val == "V1" else (1, 0) if val == "V2" else (1, 1)

def split_v1_v2(value):
    return int("V1" not in value), int("V2" not in value)


0.0025


In [29]:
# Parameter Leitungslänge [km]
ltg_l = 5

# Parameter Ausfallhäufigkeit
h_ltg110 = 0.005 * ltg_l  # [1/a]
h_ltg20 = 0.001 * ltg_l  # [1/a]
h_T = 0.015  # [1/a]
h_TR = 0.00012 + IPS  # [1/a]
h_ls = 0.000259  # [1/a]
h_SS = 0.000148  # [1/a]

# Parameter Ausfalldauer
ta_ltg110 = 80  # [h]
ta_ltg20 = 26  # [h]
ta_T = 28  # [h]
ta_TR = 72  # [h]
ta_ls = 72  # [h]
ta_SS = 72  # [h]

# Parameter Lasten [MW]
P_A = 5
P_GK = 5
P_B = 5
P_C = 5

In [30]:
# Aktuelle Situation: Umspannwerk A in H-Schaltung, der Kunde ist mit Stichleitung (20kV 5km) an den Abgang V1 des UW A angeschlossen

# Berechnung der Situation in UW A in H-Schaltung

rows = [
    # ["LTV_20kV", "LT", "V1", "V1"], # Leitung vom Abgang V1 zum Großkunden
    ["LTG1", "LT", "V1 + V2", "-"],
    ["LTG2", "LT", "V1 + V2", "-"],
    ["HTR11", "TR", "V1 + V2", "V1"],
    ["HTR12", "TR", "V1 + V2", "V2"],
    ["HTR21", "TR", "V1 + V2", "V1"],
    ["HTR22", "TR", "V1 + V2", "V2"],
    ["QTR1", "TR", "V1 + V2", "V1"],
    ["QTR2", "TR", "V1 + V2", "V2"],
    ["HLS11", "LS", "V1 + V2", "V1"],
    ["HLS12", "LS", "V1 + V2", "V2"],
    ["T1", "T", "V1", "V1"],
    ["T2", "T", "V2", "V2"],
    ["HLS21", "LS", "V1", "V1"],
    ["HLS22", "LS", "V2", "V2"],
    ["HTR31", "TR", "V1", "V1"],
    ["HTR32", "TR", "V2", "V2"],
]

elemente_H_Schaltung = [
    # {"name": "LTG_20kV", "defizithaeufigkeit": h_ltg20 * ltg_l, "ausfalldauer": ta_ltg20}, # Leitung vom Abgang V1 zum Großkunden
    {"name": "LTG1", "defizithaeufigkeit": h_ltg110, "ausfalldauer": ta_ltg110},
    {"name": "LTG2", "defizithaeufigkeit": h_ltg110, "ausfalldauer": ta_ltg110},
    {"name": "HTR11", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "HTR12", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "HTR21", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "HTR22", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "QTR1", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "QTR2", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "HLS11", "defizithaeufigkeit": h_ls, "ausfalldauer": ta_ls},
    {"name": "HLS12", "defizithaeufigkeit": h_ls, "ausfalldauer": ta_ls},
    {"name": "T1", "defizithaeufigkeit": h_T, "ausfalldauer": ta_T},
    {"name": "T2", "defizithaeufigkeit": h_T, "ausfalldauer": ta_T},
    {"name": "HLS21", "defizithaeufigkeit": h_ls, "ausfalldauer": ta_ls},
    {"name": "HLS22", "defizithaeufigkeit": h_ls, "ausfalldauer": ta_ls},
    {"name": "HTR31", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "HTR32", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
]

data = []
for row in rows:
    v1_1, v2_1 = split_v1_v2(row[2])
    v1_2, v2_2 = process_values(row[3])
    entry = next(item for item in elemente_H_Schaltung if item["name"] == row[0])
    data.append([row[0], row[1], v1_1, v2_1, v1_2, v2_2, entry["defizithaeufigkeit"], entry["ausfalldauer"]])

columns = ["Name", "Type", "V1_1", "V2_1", "V1_2", "V2_2", "Defizithäufigkeit (H in 1/a)", "Ausfalldauer (in h)"]

pd.set_option('display.max_columns', None)  # Zeigt alle Spalten an
pd.set_option('display.width', 1000)       # Erhöht die Breite für die Ausgabe
df = pd.DataFrame(data, columns=columns)

# Verbraucher 1: Wenn dieser nach der Korrektuschaltung wiederversorgt ist, muss man für NV, P und E nur die 0,25h Korrekturzeit berücksichtigen
# ansonsten muss die Ausfalldauer UND die 0,25h berücksichtigt werden
# Initialisiere die neuen Spalten mit NaN (falls sie noch nicht existieren)
df['NV_V1'] = None
df['Defizitleistung P_V1'] = None
df['Defizitenergie E_V1'] = None

H_sum_H_V1 = 0

# Defizithäufigkeit rausfiltern, die für jeweils V1 und V2 relevant sind
Defizithaeufigkeit_V1_H = (df["Defizithäufigkeit (H in 1/a)"] * (1 - df["V1_1"]))
Defizithaeufigkeit_V2_H = (df["Defizithäufigkeit (H in 1/a)"] * (1 - df["V2_1"]))

# Iteriere über alle Zeilen und berechne die neuen Werte
for index, row in df.iterrows():
    if row["V1_2"] == 0:
        df.at[index, "NV_V1"] = row["Defizithäufigkeit (H in 1/a)"] * (1 - row["V1_1"]) * (row["Ausfalldauer (in h)"] + 0.25)
        df.at[index, "Defizitleistung P_V1"] = row["Defizithäufigkeit (H in 1/a)"] * (1 - row["V1_1"]) * P_GK
        df.at[index, "Defizitenergie E_V1"] = df.at[index, "Defizitleistung P_V1"] * (row["Ausfalldauer (in h)"] + 0.25)
    elif row["V1_2"] == 1:
        df.at[index, "NV_V1"] = row["Defizithäufigkeit (H in 1/a)"] * (1 - row["V1_1"]) * 0.25
        df.at[index, "Defizitleistung P_V1"] = row["Defizithäufigkeit (H in 1/a)"] * (1 - row["V1_1"]) * P_GK
        df.at[index, "Defizitenergie E_V1"] = df.at[index, "Defizitleistung P_V1"] * 0.25


# Selbe Herangehensweise wie bei Verbraucher 2
df['NV_V2'] = None
df['Defizitleistung P_V2'] = None
df['Defizitenergie E_V2'] = None

# Iteriere über alle Zeilen und berechne die neuen Werte
for index, row in df.iterrows():
    if row["V2_2"] == 0:
        df.at[index, "NV_V2"] = row["Defizithäufigkeit (H in 1/a)"] * (1 - row["V2_1"]) * (row["Ausfalldauer (in h)"] + 0.25)
        df.at[index, "Defizitleistung P_V2"] = row["Defizithäufigkeit (H in 1/a)"] * (1 - row["V2_1"]) * P_GK
        df.at[index, "Defizitenergie E_V2"] = df.at[index, "Defizitleistung P_V2"] * (row["Ausfalldauer (in h)"] + 0.25)
    elif row["V2_2"] == 1:
        df.at[index, "NV_V2"] = row["Defizithäufigkeit (H in 1/a)"] * (1 - row["V2_1"]) * 0.25
        df.at[index, "Defizitleistung P_V2"] = row["Defizithäufigkeit (H in 1/a)"] * (1 - row["V2_1"]) * P_GK
        df.at[index, "Defizitenergie E_V2"] = df.at[index, "Defizitleistung P_V2"] * 0.25

# Summierung der Werte der jeweiligen Spalten für die H-Schaltung
Nichtverlaesslichkeit_V1_H = round(df["NV_V1"].sum(), 4)
Nichtverlaesslichkeit_V2_H = round(df["NV_V2"].sum(), 4)
Defizitleistung_V1_H = round(df["Defizitleistung P_V1"].sum(), 4)
Defizitleistung_V2_H = round(df["Defizitleistung P_V2"].sum(), 4)
Defizitenergie_V1_H = round(df["Defizitenergie E_V1"].sum(), 4)
Defizitenergie_V2_H = round(df["Defizitenergie E_V2"].sum(), 4) 
Defizithaeufigkeit_H_gesamt = round(df["Defizithäufigkeit (H in 1/a)"].sum(), 4) # fürs printen
Defizithaeufigkeit_V1_H_ges = round(Defizithaeufigkeit_V1_H.sum(),4)
Defizithaeufigkeit_V2_H_ges = round(Defizithaeufigkeit_V2_H.sum(),4)

# Exportiere als csv-Datei
df.iloc[:, :6].to_csv('output_H_Schaltung.csv', sep=';', index=False)
# display(df)
display(df.rename(columns={"Name": "Ausgefallenes Element", "Type": "Typ", "V1_1": "V1_NF", "V2_1": "V2_NF", "V1_2": "V1_NU", "V2_2": "V2_NU", "Defizithäufigkeit (H in 1/a)": 
                           "Defizithäufigkeit [1/a]", "Ausfalldauer (in h)": "Ausfalldauer [h]", "NV_V1": "Nichtverlässlichkeit V1 [h/a]", 
                           "Defizitleistung P_V1": "Defizitleistung V1 [MW/a]", "Defizitenergie E_V1": "Defizitenergie V1 [MWh/a]", "NV_V2": "Nichtverlässlichkeit V2 [h/a]", 
                           "Defizitleistung P_V2": "Defizitleistung V2 [MW/a]", "Defizitenergie E_V2": "Defizitenergie V2 [MWh/a]"}))
print(" ")

# print(H_sum_H_V1)
# print(Defizithaeufigkeit_V1_H_gesamt)

# Ausgeben der summierten Werte
print(f"Defizithäufigkeit V1 gesamt (H): {Defizithaeufigkeit_V1_H_ges} [1/a]")
print(f"Defizithäufigkeit V2 gesamt (H): {Defizithaeufigkeit_V2_H_ges} [1/a]")
print(f"Nichtverlässlichkeit V1 (H): {Nichtverlaesslichkeit_V1_H} h/a = {round(Nichtverlaesslichkeit_V1_H / 8760, 6)} [1]")
print(f"Nichtverlässlichkeit V2 (H): {Nichtverlaesslichkeit_V2_H} h/a = {round(Nichtverlaesslichkeit_V2_H / 8760, 6)} [1]")
print(f"Defizitleistung V1 (H): {Defizitleistung_V1_H} MW")
print(f"Defizitleistung V2 (H): {Defizitleistung_V2_H} MW")
print(f"Defizitenergie V1 (H): {Defizitenergie_V1_H} MWh")
print(f"Defizitenergie V2 (H): {Defizitenergie_V2_H} MWh")
print(" ")
print(f"Nichtverlässlichkeit Gesamt (H): {Nichtverlaesslichkeit_V1_H + Nichtverlaesslichkeit_V2_H} h/a")
print(f"Defizitleistung Gesamt (H): {Defizitleistung_V1_H + Defizitleistung_V2_H} MW")
print(f"Defizitenergie Gesamt (H): {Defizitenergie_V1_H + Defizitenergie_V2_H} MWh")

,Ausgefallenes Element,Typ,V1_NF,V2_NF,V1_NU,V2_NU,Defizithäufigkeit [1/a],Ausfalldauer [h],Nichtverlässlichkeit V1 [h/a],Defizitleistung V1 [MW/a],Defizitenergie V1 [MWh/a],Nichtverlässlichkeit V2 [h/a],Defizitleistung V2 [MW/a],Defizitenergie V2 [MWh/a]
0,LTG1,LT,0,0,1,1,0.025000,80,0.00625,0.125,0.03125,0.00625,0.125,0.03125
1,LTG2,LT,0,0,1,1,0.025000,80,0.00625,0.125,0.03125,0.00625,0.125,0.03125
2,HTR11,TR,0,0,0,1,0.002620,72,0.189295,0.0131,0.946475,0.000655,0.0131,0.003275
3,HTR12,TR,0,0,1,0,0.002620,72,0.000655,0.0131,0.003275,0.189295,0.0131,0.946475
4,HTR21,TR,0,0,0,1,0.002620,72,0.189295,0.0131,0.946475,0.000655,0.0131,0.003275
5,HTR22,TR,0,0,1,0,0.002620,72,0.000655,0.0131,0.003275,0.189295,0.0131,0.946475
6,QTR1,TR,0,0,0,1,0.002620,72,0.189295,0.0131,0.946475,0.000655,0.0131,0.003275
7,QTR2,TR,0,0,1,0,0.002620,72,0.000655,0.0131,0.003275,0.189295,0.0131,0.946475
8,HLS11,LS,0,0,0,1,0.000259,72,0.018713,0.001295,0.093564,0.000065,0.001295,0.000324
9,HLS12,LS,0,0,1,0,0.000259,72,0.000065,0.001295,0.000324,0.018713,0.001295,0.093564


 
Defizithäufigkeit V1 gesamt (H): 0.0841 [1/a]
Defizithäufigkeit V2 gesamt (H): 0.0841 [1/a]
Nichtverlässlichkeit V1 (H): 1.2329 h/a = 0.000141 [1]
Nichtverlässlichkeit V2 (H): 1.2329 h/a = 0.000141 [1]
Defizitleistung V1 (H): 0.4206 MW
Defizitleistung V2 (H): 0.4206 MW
Defizitenergie V1 (H): 6.1644 MWh
Defizitenergie V2 (H): 6.1644 MWh
 
Nichtverlässlichkeit Gesamt (H): 2.4658 h/a
Defizitleistung Gesamt (H): 0.8412 MW
Defizitenergie Gesamt (H): 12.3288 MWh


In [31]:
# Berechnung für Doppelsammelschienen-Topologie

elemente_DSS_Schaltung = [
    {"name": "LTG1", "defizithaeufigkeit": h_ltg110, "ausfalldauer": ta_ltg110},
    {"name": "LTG2", "defizithaeufigkeit": h_ltg110, "ausfalldauer": ta_ltg110},
    {"name": "LTR11", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "LTR12", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "LS11", "defizithaeufigkeit": h_ls, "ausfalldauer": ta_ls},
    {"name": "LS12", "defizithaeufigkeit": h_ls, "ausfalldauer": ta_ls},
    {"name": "STR11", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "STR12", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "STR21", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "STR22", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "QTR1", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "QTR2", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "QLS", "defizithaeufigkeit": h_ls, "ausfalldauer": ta_ls},
    {"name": "SS1", "defizithaeufigkeit": h_SS, "ausfalldauer": ta_SS},
    {"name": "SS2", "defizithaeufigkeit": h_SS, "ausfalldauer": ta_SS},
    {"name": "STR31", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "STR32", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "STR41", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "STR42", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "LS21", "defizithaeufigkeit": h_ls, "ausfalldauer": ta_ls},
    {"name": "LS22", "defizithaeufigkeit": h_ls, "ausfalldauer": ta_ls},
    {"name": "T1", "defizithaeufigkeit": h_T, "ausfalldauer": ta_T},
    {"name": "T2", "defizithaeufigkeit": h_T, "ausfalldauer": ta_T},
    {"name": "LS31", "defizithaeufigkeit": h_ls, "ausfalldauer": ta_ls},
    {"name": "LS32", "defizithaeufigkeit": h_ls, "ausfalldauer": ta_ls},
    {"name": "ATR11", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
    {"name": "ATR12", "defizithaeufigkeit": h_TR, "ausfalldauer": ta_TR},
]

# Beschreibung der Ausfallwirkung für die Doppelsammelschienen-Schaltanlage in einer Tabelle
rows = []

# Der in der Abbildung ersichtliche Schaltzustand wird als Basis für nachfolgende Überlegungen getroffen

# Zeilenweise Eintragen der Werte
rows.append(["LTG1", "LT", "-", "-"])
rows.append(["LTG2", "LT", "-", "-"])
rows.append(["LTR11", "TR", "-", "-"])
rows.append(["LTR12", "TR", "-", "-"])
rows.append(["LS11", "LS", "V1 + V2", "-"])
rows.append(["LS12", "LS", "V1 + V2", "-"])

rows.append(["STR11", "TR", "V1 + V2", "-"])
rows.append(["STR12", "TR", "V1 + V2", "-"]) 

rows.append(["STR21", "TR", "V1 + V2", "-"]) 
rows.append(["STR22", "TR", "V1 + V2", "-"]) 

rows.append(["QTR1", "TR", "V1 + V2", "-"])
rows.append(["QTR2", "TR", "V1 + V2", "-"])
rows.append(["QLS", "LS", "V1 + V2", "-"])

rows.append(["SS1", "SS", "V1 + V2", "-"])
rows.append(["SS2", "SS", "V1 + V2", "-"])

rows.append(["STR31", "TR", "V1 + V2", "V1"])
rows.append(["STR32", "TR", "V1 + V2", "V2"])

rows.append(["STR41", "TR", "V1 + V2", "V1"])
rows.append(["STR42", "TR", "V1 + V2", "V2"])

rows.append(["LS21", "LS", "V1 + V2", "V1"])
rows.append(["LS22", "LS", "V1 + V2", "V2"])

rows.append(["T1", "T", "V1", "V1"])
rows.append(["T2", "T", "V2", "V2"])

rows.append(["LS31", "LS", "V1", "V1"])
rows.append(["LS32", "LS", "V2", "V2"])

rows.append(["ATR11", "TR", "V1", "V1"])
rows.append(["ATR12", "TR", "V2", "V2"])

# Selbes Procedere wie bei H-Schaltung
data = []
for row in rows:
    v1_1, v2_1 = split_v1_v2(row[2])
    v1_2, v2_2 = process_values(row[3])
    entry = next(item for item in elemente_DSS_Schaltung if item["name"] == row[0])
    data.append([row[0], row[1], v1_1, v2_1, v1_2, v2_2, entry["defizithaeufigkeit"], entry["ausfalldauer"]])

columns = ["Name", "Type", "V1_1", "V2_1", "V1_2", "V2_2", "Defizithäufigkeit (H in 1/a)", "Ausfalldauer (in h)"]

pd.set_option('display.max_columns', None)  # Zeigt alle Spalten an
pd.set_option('display.width', 1000)       # Erhöht die Breite für die Ausgabe
df = pd.DataFrame(data, columns=columns)

# Verbraucher 1: Wenn dieser nach der Korrektuschaltung wiederversorgt ist, muss man für NV, P und E nur die 0,25h Korrekturzeit berücksichtigen
# ansonsten muss die Ausfalldauer UND die 0,25h berücksichtigt werden
# Initialisiere die neuen Spalten mit NaN (falls sie noch nicht existieren)
df['NV_V1'] = None
df['Defizitleistung P_V1'] = None
df['Defizitenergie E_V1'] = None

# Defizithäufigkeit rausfiltern, die für jeweils V1 und V2 relevant sind
Defizithaeufigkeit_V1_DSS = (df["Defizithäufigkeit (H in 1/a)"] * (1 - df["V1_1"]))
Defizithaeufigkeit_V2_DSS = (df["Defizithäufigkeit (H in 1/a)"] * (1 - df["V2_1"]))


# Iteriere über alle Zeilen und berechne die neuen Werte
for index, row in df.iterrows():
    if row["V1_2"] == 0:
        df.at[index, "NV_V1"] = row["Defizithäufigkeit (H in 1/a)"] * (1 - row["V1_1"]) * (row["Ausfalldauer (in h)"] + 0.25)
        df.at[index, "Defizitleistung P_V1"] = row["Defizithäufigkeit (H in 1/a)"] * (1 - row["V1_1"]) * P_GK
        df.at[index, "Defizitenergie E_V1"] = df.at[index, "Defizitleistung P_V1"] * (row["Ausfalldauer (in h)"] + 0.25)
    elif row["V1_2"] == 1:
        df.at[index, "NV_V1"] = row["Defizithäufigkeit (H in 1/a)"] * (1 - row["V1_1"]) * 0.25
        df.at[index, "Defizitleistung P_V1"] = row["Defizithäufigkeit (H in 1/a)"] * (1 - row["V1_1"]) * P_GK
        df.at[index, "Defizitenergie E_V1"] = df.at[index, "Defizitleistung P_V1"] * 0.25


# Selbe Herangehensweise wie bei Verbraucher 2
df['NV_V2'] = None
df['Defizitleistung P_V2'] = None
df['Defizitenergie E_V2'] = None

# Iteriere über alle Zeilen und berechne die neuen Werte
for index, row in df.iterrows():
    if row["V2_2"] == 0:
        df.at[index, "NV_V2"] = row["Defizithäufigkeit (H in 1/a)"] * (1 - row["V2_1"]) * (row["Ausfalldauer (in h)"] + 0.25)
        df.at[index, "Defizitleistung P_V2"] = row["Defizithäufigkeit (H in 1/a)"] * (1 - row["V2_1"]) * P_GK
        df.at[index, "Defizitenergie E_V2"] = df.at[index, "Defizitleistung P_V2"] * (row["Ausfalldauer (in h)"] + 0.25)
    elif row["V2_2"] == 1:
        df.at[index, "NV_V2"] = row["Defizithäufigkeit (H in 1/a)"] * (1 - row["V2_1"]) * 0.25
        df.at[index, "Defizitleistung P_V2"] = row["Defizithäufigkeit (H in 1/a)"] * (1 - row["V2_1"]) * P_GK
        df.at[index, "Defizitenergie E_V2"] = df.at[index, "Defizitleistung P_V2"] * 0.25

# Summierung der Werte der jeweiligen Spalten für die H-Schaltung
Nichtverlaesslichkeit_V1_DSS = round(df["NV_V1"].sum(), 4)
Nichtverlaesslichkeit_V2_DSS = round(df["NV_V2"].sum(), 4)
Defizitleistung_V1_DSS = round(df["Defizitleistung P_V1"].sum(), 4)
Defizitleistung_V2_DSS = round(df["Defizitleistung P_V2"].sum(), 4)
Defizitenergie_V1_DSS = round(df["Defizitenergie E_V1"].sum(), 4)
Defizitenergie_V2_DSS = round(df["Defizitenergie E_V2"].sum(), 4)
Defizithaeufigkeit_H_gesamt_DSS = df["Defizithäufigkeit (H in 1/a)"].sum()
Defizithaeufigkeit_V1_DSS_ges = round(Defizithaeufigkeit_V1_DSS.sum(),4)
Defizithaeufigkeit_V2_DSS_ges = round(Defizithaeufigkeit_V2_DSS.sum(),4)
# Exportiere als csv-Datei
df.iloc[:, :6].to_csv('output_DSS_Schaltung.csv', sep=';', index=False)
# display(df)
display(df.rename(columns={"Name": "Ausgefallenes Element", "Type": "Typ", "V1_1": "V1_NF", "V2_1": "V2_NF", "V1_2": "V1_NU", "V2_2": "V2_NU", "Defizithäufigkeit (H in 1/a)": 
                           "Defizithäufigkeit [1/a]", "Ausfalldauer (in h)": "Ausfalldauer [h]", "NV_V1": "Nichtverlässlichkeit V1 [h/a]", 
                           "Defizitleistung P_V1": "Defizitleistung V1 [MW/a]", "Defizitenergie E_V1": "Defizitenergie V1 [MWh/a]", "NV_V2": "Nichtverlässlichkeit V2 [h/a]", 
                           "Defizitleistung P_V2": "Defizitleistung V2 [MW/a]", "Defizitenergie E_V2": "Defizitenergie V2 [MWh/a]"}))
print(" ")

# Ausgeben der summierten Werte
print(f"Defizithäufigkeit V1 gesamt (DSS): {Defizithaeufigkeit_V1_DSS_ges} [1/a]")
print(f"Defizithäufigkeit V2 gesamt (DSS): {Defizithaeufigkeit_V2_DSS_ges} [1/a]")
print(f"Nichtverlässlichkeit V1 (DSS): {Nichtverlaesslichkeit_V1_DSS} h/a = {round(Nichtverlaesslichkeit_V1_DSS / 8760, 6)} [1]")
print(f"Nichtverlässlichkeit V2 (DSS): {Nichtverlaesslichkeit_V2_DSS} h/a = {round(Nichtverlaesslichkeit_V2_DSS / 8760, 6)} [1]")
print(f"Defizitleistung V1 (DSS): {Defizitleistung_V1_DSS} MW")
print(f"Defizitleistung V2 (DSS): {Defizitleistung_V2_DSS} MW")
print(f"Defizitenergie V1 (DSS): {Defizitenergie_V1_DSS} MWh")
print(f"Defizitenergie V2 (DSS): {Defizitenergie_V2_DSS} MWh")
print(" ")
print(f"Nichtverlässlichkeit Gesamt (DSS): {Nichtverlaesslichkeit_V1_DSS + Nichtverlaesslichkeit_V2_DSS} h/a")
print(f"Defizitleistung Gesamt (DSS): {Defizitleistung_V1_DSS + Defizitleistung_V2_DSS} MW")
print(f"Defizitenergie Gesamt (DSS): {Defizitenergie_V1_DSS + Defizitenergie_V2_DSS} MWh")

,Ausgefallenes Element,Typ,V1_NF,V2_NF,V1_NU,V2_NU,Defizithäufigkeit [1/a],Ausfalldauer [h],Nichtverlässlichkeit V1 [h/a],Defizitleistung V1 [MW/a],Defizitenergie V1 [MWh/a],Nichtverlässlichkeit V2 [h/a],Defizitleistung V2 [MW/a],Defizitenergie V2 [MWh/a]
0,LTG1,LT,1,1,1,1,0.025000,80,0.0,0.0,0.0,0.0,0.0,0.0
1,LTG2,LT,1,1,1,1,0.025000,80,0.0,0.0,0.0,0.0,0.0,0.0
2,LTR11,TR,1,1,1,1,0.002620,72,0.0,0.0,0.0,0.0,0.0,0.0
3,LTR12,TR,1,1,1,1,0.002620,72,0.0,0.0,0.0,0.0,0.0,0.0
4,LS11,LS,0,0,1,1,0.000259,72,0.000065,0.001295,0.000324,0.000065,0.001295,0.000324
5,LS12,LS,0,0,1,1,0.000259,72,0.000065,0.001295,0.000324,0.000065,0.001295,0.000324
6,STR11,TR,0,0,1,1,0.002620,72,0.000655,0.0131,0.003275,0.000655,0.0131,0.003275
7,STR12,TR,0,0,1,1,0.002620,72,0.000655,0.0131,0.003275,0.000655,0.0131,0.003275
8,STR21,TR,0,0,1,1,0.002620,72,0.000655,0.0131,0.003275,0.000655,0.0131,0.003275
9,STR22,TR,0,0,1,1,0.002620,72,0.000655,0.0131,0.003275,0.000655,0.0131,0.003275


 
Defizithäufigkeit V1 gesamt (DSS): 0.0457 [1/a]
Defizithäufigkeit V2 gesamt (DSS): 0.0457 [1/a]
Nichtverlässlichkeit V1 (DSS): 1.0346 h/a = 0.000118 [1]
Nichtverlässlichkeit V2 (DSS): 1.0346 h/a = 0.000118 [1]
Defizitleistung V1 (DSS): 0.2283 MW
Defizitleistung V2 (DSS): 0.2283 MW
Defizitenergie V1 (DSS): 5.1732 MWh
Defizitenergie V2 (DSS): 5.1732 MWh
 
Nichtverlässlichkeit Gesamt (DSS): 2.0692 h/a
Defizitleistung Gesamt (DSS): 0.4566 MW
Defizitenergie Gesamt (DSS): 10.3464 MWh


In [32]:
# Parameter für die 20kV Leitung bestimmen 
NV_ltg20 = h_ltg20 * ta_ltg20 # in [h/a], später in h/h angeben
P_ltg20 = h_ltg20 * P_GK # in [MW]
E_ltg20 = P_ltg20 * ta_ltg20 # in [MWh]

print(f"NV 20kV: {NV_ltg20}")
print(f"P 20kV: {P_ltg20}")
print(f"E 20kV: {E_ltg20}")

# Fall 1 Aktuelle Situation: Großkunde ist über die Stichleitung mit 20kV an V1 von UWA angebunden. Daraus lässt sich der gesamte Block
# aus einer Serienschaltung von UWA und 20kV-Leitung angeben
H_Gesamt_Fall1 = Defizithaeufigkeit_V1_H_ges + h_ltg20 # in 1/a
NV_Gesamt_Fall1 = (Nichtverlaesslichkeit_V1_H + NV_ltg20) # in h/a
lambda_Fall1 = H_Gesamt_Fall1 / (1 - NV_Gesamt_Fall1 / 8760)
mu_Fall1 = H_Gesamt_Fall1 / (NV_Gesamt_Fall1 / 8760)

print("Lambda Fall 1:", lambda_Fall1)
print("mu Fall 1:", mu_Fall1)

# Zustandsmatrix aufstellen
zustandsmatrix_Fall1 = np.array([[1 - lambda_Fall1, lambda_Fall1], [mu_Fall1, 1 - mu_Fall1]])

print("Zustandsmatrix Fall 1:")
print(zustandsmatrix_Fall1)

# Startvektor p(0): Zu Beginn alles in Betrieb und nicht ausgefallen
b = np.array([1, 0])

# Berechnung von p
pFall1 = zustandsmatrix_Fall1.T @ b
print(f"Wahrscheinlichkeitsvektor nach einem Schritt für Fall 1: {np.round(pFall1, 4)}")

# Fall 2: Großkunde wird nun zusätzlich mit einer weiteren Stichleitung an UWB angeschlossen. Hier kann man nun folgende Blöcke definieren:
# BLOCK A: UWA + 20kV-Leitung
# BLOCK B: UWB + 20kV Leitung + 20kV Leitung

# Werte für BLOCK A bestimmen:
H_BLOCK_A = Defizithaeufigkeit_V1_H.sum() + h_ltg20 # in 1/a
NV_BLOCK_A = (Nichtverlaesslichkeit_V1_H + NV_ltg20) # in h/a
lambda_BLOCK_A = H_BLOCK_A / (1 - NV_BLOCK_A / 8760)
mu_BLOCK_A = H_BLOCK_A / (NV_BLOCK_A / 8760)

print("H Block A", H_BLOCK_A)
print("NV Block A", NV_BLOCK_A)
print(f"lambda BLOCK A: {lambda_BLOCK_A}")
print(f"mu BLOCK A: {mu_BLOCK_A}")

# Werte für BLOCK B bestimmen:
H_BLOCK_B = Defizithaeufigkeit_V1_DSS.sum() + 2 * h_ltg20 # in 1/a
NV_BLOCK_B = (Nichtverlaesslichkeit_V1_DSS + 2 * NV_ltg20) # in h/a
lambda_BLOCK_B = H_BLOCK_B / (1 - NV_BLOCK_B / 8760)
mu_BLOCK_B = H_BLOCK_B / (NV_BLOCK_B / 8760)

print("H Block B", H_BLOCK_B)
print("NV Block B", NV_BLOCK_B)
print(f"lambda BLOCK B: {lambda_BLOCK_B}")
print(f"mu BLOCK B: {mu_BLOCK_B}")

# Zustandsmatrix für Fall 2 aufstellen
'''zustandsmatrix_Fall2= np.array([
    [-(lambda_BLOCK_A+lambda_BLOCK_B),  mu_BLOCK_A,                             mu_BLOCK_A,                          mu_BLOCK_B,                         0],
    [lambda_BLOCK_A,                    -(mu_BLOCK_A+1/(50/60)+lambda_BLOCK_B), 0,                                   0,                                  mu_BLOCK_B],
    [0,                                 1/(50/60),                              -(lambda_BLOCK_B+mu_BLOCK_A),        0,                                  mu_BLOCK_B],
    [lambda_BLOCK_B,                    0,                                      0,                                   -(mu_BLOCK_B + lambda_BLOCK_A),     mu_BLOCK_A],
    [0,                                 lambda_BLOCK_B,                         lambda_BLOCK_B,                     lambda_BLOCK_A,                     -(mu_BLOCK_B+mu_BLOCK_B+mu_BLOCK_A)]
])'''

zustandsmatrix_Fall2= np.array([
    [1,                                 1,                                      1,                                   1,                                  1],
    [lambda_BLOCK_A,                    -(mu_BLOCK_A+1/(50/60)+lambda_BLOCK_B), 0,                                   0,                                  mu_BLOCK_B],
    [0,                                 1/(50/60),                              -(lambda_BLOCK_B+mu_BLOCK_A),        0,                                  mu_BLOCK_B],
    [lambda_BLOCK_B,                    0,                                      0,                                   -(mu_BLOCK_B + lambda_BLOCK_A),     mu_BLOCK_A],
    [0,                                 lambda_BLOCK_B,                         lambda_BLOCK_B,                     lambda_BLOCK_A,                     -(mu_BLOCK_B+mu_BLOCK_B+mu_BLOCK_A)]
])


"""zustandsmatrix_Fall2= np.array([
    [1,  1,                             1,                          1,                         1],
    [lambda_BLOCK_A,                    -(1/(0.25)+1/(50/60)+lambda_BLOCK_B), 0,                                   0,                                  mu_BLOCK_B],
    [0,                                 1/(50/60),                              -(lambda_BLOCK_B+1/72),        0,                                  mu_BLOCK_B],
    [lambda_BLOCK_B,                    0,                                      0,                                   -(mu_BLOCK_B + lambda_BLOCK_A),     mu_BLOCK_A],
    [0,                                 lambda_BLOCK_B,                         lambda_BLOCK_B,                     lambda_BLOCK_A,                     -(mu_BLOCK_B+mu_BLOCK_B+mu_BLOCK_A)]
])""" # für Alex und alle interessierten ;)

np.set_printoptions(precision=5, suppress=True)  # Rundet auf 3 Nachkommastellen
print("Zustandsmatrix Fall 2:")
print(zustandsmatrix_Fall2)

# Startvektor p(0): Zu Beginn alles in Betrieb und nicht ausgefallen
b = np.array([1, 0, 0, 0, 0])
pFall2 = np.linalg.solve(zustandsmatrix_Fall2, b)

# Berechnung von p
# pFall2 = zustandsmatrix_Fall2 @ b
# pFall2 = zustandsmatrix_Fall2.T @ b

# print()
print(f"Wahrscheinlichkeitsvektor nach einem Schritt für Fall 2: {pFall2}")



# Fall 3: Das UWA wird auf eine Doppelsammelschienentopologie erweitert
H_Gesamt_FallC = Defizithaeufigkeit_V1_DSS.sum() + h_ltg20 # in 1/a
NV_Gesamt_FallC = (Nichtverlaesslichkeit_V1_DSS + NV_ltg20) # in h/a
lambda_FallC = H_Gesamt_FallC / (1 - NV_Gesamt_FallC / 8760)
mu_FallC = H_Gesamt_FallC / (NV_Gesamt_FallC / 8760)

#print(H_Gesamt_FallC)
#print(NV_Gesamt_FallC)
#print(lambda_FallC)
#print(mu_FallC)

# print(NV_Gesamt_FallC)

# Zustandsmatrix aufstellen
zustandsmatrix_Fall3 = np.array([[1 - lambda_FallC, lambda_FallC], [mu_FallC, 1 - mu_FallC]])
print("Zustandsmatrix Fall 3:")
print(zustandsmatrix_Fall3)

# Startvektor p(0): Zu Beginn alles in Betrieb und nicht ausgefallen
b = np.array([1, 0])

# Berechnung von p
pFall3 = zustandsmatrix_Fall3.T @ b



print(f"Wahrscheinlichkeitsvektor nach einem Schritt für Fall 3: {pFall3}")


NV 20kV: 0.13
P 20kV: 0.025
E 20kV: 0.65
Lambda Fall 1: 0.08911386453036169
mu Fall 1: 572.687651331719
Zustandsmatrix Fall 1:
[[   0.91089    0.08911]
 [ 572.68765 -571.68765]]
Wahrscheinlichkeitsvektor nach einem Schritt für Fall 1: [0.9109 0.0891]
H Block A 0.089117
NV Block A 1.3629000000000002
lambda BLOCK A: 0.08913086717567052
mu BLOCK A: 572.7969183359013
H Block B 0.05567
NV Block B 1.2946
lambda BLOCK B: 0.05567822842859859
mu BLOCK B: 376.69488645141354
Zustandsmatrix Fall 2:
[[    1.          1.          1.          1.          1.     ]
 [    0.08913  -574.0526      0.          0.        376.69489]
 [    0.          1.2      -572.8526      0.        376.69489]
 [    0.05568     0.          0.       -376.78402   572.79692]
 [    0.          0.05568     0.05568     0.08913 -1326.18669]]
Wahrscheinlichkeitsvektor nach einem Schritt für Fall 2: [0.9997  0.00016 0.      0.00015 0.     ]
Zustandsmatrix Fall 3:
[[   0.94932    0.05068]
 [ 381.13447 -380.13447]]
Wahrscheinlichkeits